## $\S$2. CNN

### 2.1 CNN网络

#### 一. 计算机视觉相关问题
1. 用于计算机视觉的神经网络, 其每层节点的作用不同: 如  
  1. 第一层网络检测物体边界
  2. 后面的网络检测出物体
  3. 最后面的网络检测出完整的物体, 如一张完整的人脸

2. 如下一张灰度图, 我们要检测出他的垂直边界.   
  1. 灰度图使用6\*6\*1的矩阵表示 (因为不是rgb图像, 所以不是6\*6\*3)  
  2. 我们定义一个3\*3的"过滤器"矩阵filter, 该矩阵也称为"核"  
  3. 使用灰度值矩阵和filter进行卷积操作  
   中间的星号代表卷积操作 : 将filter放在灰度值矩阵的顶端, 把相对应的元素进行乘积求和, 作为结果矩阵的第一个元素.   
   然后移动filter一列, 和对应的弧度制矩阵做相同的乘积求和(卷积)操作, 结果作为第二个元素. 以此类推 ..   
       <img src="../img/edgedetection">  
3. 卷积操作的编程实现  
  1. tensorflow中 : `tf.nn.conv2d`
  2. keras中 : `Conv2D`
  
4. 为了阐述卷积操作如何进行边沿检测, 我们使用一个简单的例子  
  1. 使用左侧全为10, 右侧全为0的灰度值矩阵, filter使用3\*3的矩阵  
  2. 若0表示灰色, 正值表示白色, 负值表示黑色. 则卷积操作的结果为"中间白色, 两边灰色的图像". 中间的白色条即为检测出的垂直边界
  <img src="../img/edgedetection2.png" height="60%" width="60%">